In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import joblib

In [2]:
data = pd.read_parquet('../data/ML_data/ML_train_data.parquet')
data.drop(['Station_Name', 'Business_Date', 'PublicHoliday', 'Date'], axis=1, inplace=True)
y = data['log_Total_Demand']
X = data.drop(['log_Total_Demand'], axis=1)


In [3]:
data

,Weekday,log_Total_Demand,mean_rainfall_value,has_school,has_sport_facility,has_shopping_centre,has_hospital,total_population,med_rent_weekly_c2021,med_mortg_rep_mon_c2021,...,Westall,Westgarth,Westona,Williams Landing,Williamstown,Williamstown Beach,Willison,Windsor,Yarraman,Yarraville
0,0,-1.590078,-1.556931,0,0,0,0,-0.829089,-0.355036,-0.459351,...,-1.251021,-0.997419,1.030019,2.494975,0.505772,0.767413,-1.412212,-0.841184,-0.978305,0.307137
1,0,-2.677412,-1.556931,0,0,0,0,-0.829089,-0.355036,-0.459351,...,-1.252371,-0.986387,1.528683,2.569257,0.557268,0.844431,-1.412212,-0.811846,-0.986107,0.304484
2,1,-2.290096,-1.556931,0,0,0,0,-0.829089,-0.355036,-0.459351,...,-1.248455,-1.722069,1.613901,2.630175,0.547518,0.642515,-1.412212,-0.876631,-0.990132,0.327493
3,1,-1.676214,-1.556931,0,0,0,0,-0.829089,-0.355036,-0.459351,...,-1.239132,-1.722069,1.572998,2.904756,0.483018,0.679989,-1.141728,-0.781095,-0.986107,0.398527
4,1,-1.449649,-1.556931,0,0,0,0,-0.829089,-0.355036,-0.459351,...,-1.240119,-1.722069,1.572998,2.829185,0.172741,0.515339,-1.141728,-0.777384,-0.981542,0.392771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79949,0,0.697757,-1.175627,1,1,0,0,0.229075,0.510369,0.581919,...,-0.471748,0.740544,1.716632,1.735370,2.091581,1.552048,-0.651573,0.798081,-0.677804,-1.621478
79950,0,0.595880,-1.175627,1,1,0,0,0.229075,0.510369,0.581919,...,-0.494365,0.654487,1.654215,1.668574,1.539415,1.674669,-0.119909,0.628862,-0.693987,-1.621478
79951,1,0.652032,-1.175627,1,1,0,0,0.229075,0.510369,0.581919,...,-0.485176,0.624070,1.784640,1.949590,1.511881,1.633596,0.264328,0.704559,-0.717370,-1.621478
79952,1,0.733820,-1.175627,1,1,0,0,0.229075,0.510369,0.581919,...,-0.481339,0.700083,1.891953,2.032609,1.630725,1.606028,0.354159,0.765183,-0.739428,-1.621478


In [3]:
validation = pd.read_parquet('../data/ML_data/ML_val_data.parquet')
validation.drop(['Station_Name', 'Business_Date', 'PublicHoliday', 'Date'], axis=1, inplace=True)
y_val = validation['log_Total_Demand']
X_val = validation.drop(['log_Total_Demand'], axis=1)


### Linear Regression

In [88]:
def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.05,
                       threshold_out = 0.05,
                       verbose=True):
    """
    Perform a forward-backward feature selection
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns:
        - list of selected features
        - fitted model with selected features
    """
    included = list(initial_list)
    while True:
        changed=False

        # Forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype='float64')
        for new_column in excluded:
            model = sm.OLS(y, (pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print(f'Added {best_feature} with p-value {best_pval:.6}')

        # Backward step
        model = sm.OLS(y, (pd.DataFrame(X[included]))).fit()
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print(f'Dropped {worst_feature} with p-value {worst_pval:.6}')
        if not changed:
            break

    # Fit the model with the final selected features
    final_model = sm.OLS(y, (pd.DataFrame(X[included]))).fit()

    return included, final_model

X_const = sm.add_constant(X)
X_val_const = sm.add_constant(X_val)
selected_features, final_model = stepwise_selection(X_const, y)

Added Jacana with p-value 0.0
Added West Footscray with p-value 0.0
Added Tooronga with p-value 0.0
Added Officer with p-value 0.0
Added Wattle Glen with p-value 0.0
Added Williams Landing with p-value 0.0
Added Mordialloc with p-value 0.0
Added Aircraft with p-value 0.0
Added Mooroolbark with p-value 0.0
Added Crib Point with p-value 0.0
Added Frankston with p-value 0.0
Dropped West Footscray with p-value 0.381213
Added Southern Cross with p-value 0.0
Added Aspendale with p-value 0.0
Added has_hospital with p-value 0.0
Dropped Mooroolbark with p-value 0.058723
Added Upfield with p-value 0.0
Added  med_mortg_rep_mon_c2021 with p-value 0.0
Added  med_person_inc_we_c2021 with p-value 0.0
Added  med_famly_inc_we_c2021 with p-value 0.0
Added total_population with p-value 0.0
Added Flinders Street with p-value 0.0
Dropped  med_mortg_rep_mon_c2021 with p-value 0.462761
Added Hastings with p-value 0.0
Added Richmond with p-value 0.0
Added Collingwood with p-value 0.0
Added mean_rainfall_value

In [89]:
print(final_model.summary())

                            OLS Regression Results                            
Dep. Variable:       log_Total_Demand   R-squared:                       0.759
Model:                            OLS   Adj. R-squared:                  0.758
Method:                 Least Squares   F-statistic:                     1232.
Date:                Sat, 25 May 2024   Prob (F-statistic):               0.00
Time:                        23:05:53   Log-Likelihood:                -56552.
No. Observations:               79954   AIC:                         1.135e+05
Df Residuals:                   79749   BIC:                         1.154e+05
Df Model:                         204                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Jacana                  

In [93]:
import pickle
with open('../output/ols_step_model_05.pkl', 'wb') as f:
    pickle.dump(final_model, f)

In [90]:
y_train_pred = final_model.predict(X_const[list(final_model.params.index)])
mse_ols = mean_squared_error(y, y_train_pred)
print(f"OLS Train Mean Squared Error: {mse_ols}")

OLS Train Mean Squared Error: 0.24092428021551174


In [91]:

y_pred_ols = final_model.predict(X_val_const[list(final_model.params.index)])
mse_ols = mean_squared_error(y_val, y_pred_ols)
print(f"OLS Validation Mean Squared Error: {mse_ols}")

OLS Validation Mean Squared Error: 0.26344487185087373


In [30]:
import pickle
with open('../output/ols_step_model_05.pkl', 'rb') as file:
    LR = pickle.load(file)
LR_result = pd.DataFrame([LR.params, LR.tvalues, LR.pvalues], index=['coef', 't', 'p-value']).T
LR_result.to_csv('../output/LR_estimate.csv')

In [35]:
test = pd.read_parquet('../data/ML_data/ML_test_data.parquet')
test.drop(['Station_Name', 'Business_Date', 'PublicHoliday', 'Date'], axis=1, inplace=True)
y_test = test['log_Total_Demand']
X_test = test.drop(['log_Total_Demand'], axis=1)
X_val_const = sm.add_constant(X_val)
X_const = sm.add_constant(X)
X_test_const = sm.add_constant(X_test)
y_pred_train = LR.predict(X_const[list(LR.params.index)])
y_pred_val = LR.predict(X_val_const[list(LR.params.index)])
y_pred_test = LR.predict(X_test_const[list(LR.params.index)])

print(f"Training Mean Squared Error: {round(mean_squared_error(y, y_pred_train),4)}, R-Squared = {round(r2_score(y, y_pred_train),4)}")
print(f"Validation Mean Squared Error: {round(mean_squared_error(y_val, y_pred_val),4)}, R-Squared = {round(r2_score(y_val, y_pred_val),4)}")
print(f"Test Mean Squared Error: {round(mean_squared_error(y_test, y_pred_test),4)}, R-Squared = {round(r2_score(y_test, y_pred_test),4)}")

Training Mean Squared Error: 0.2409, R-Squared = 0.7591
Validation Mean Squared Error: 0.2634, R-Squared = 0.7359
Test Mean Squared Error: 0.583, R-Squared = 0.3495


#### XGB

In [32]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
model = xgb.XGBRegressor(objective='reg:squarederror', max_depth=3, n_estimators=150)
model.fit(X, y)
joblib.dump(model, '../output/xgb_model.pkl')
y_pred_train = model.predict(X)
y_pred_val = model.predict(X_val)
y_pred_test = model.predict(X_test)


print(f"Training Mean Squared Error: {round(mean_squared_error(y, y_pred_train),4)}, R-Squared = {round(r2_score(y, y_pred_train),4)}")
print(f"Validation Mean Squared Error: {round(mean_squared_error(y_val, y_pred_val),4)}, R-Squared = {round(r2_score(y_val, y_pred_val),4)}")
print(f"Test Mean Squared Error: {round(mean_squared_error(y_test, y_pred_test),4)}, R-Squared = {round(r2_score(y_test, y_pred_test),4)}")

Training Mean Squared Error: 0.084, R-Squared = 0.916
Validation Mean Squared Error: 0.1081, R-Squared = 0.8916
Test Mean Squared Error: 0.1519, R-Squared = 0.8305


### LGBM

In [27]:
import lightgbm as lgb

model = lgb.LGBMRegressor(objective='regression', max_depth=10, num_leaves=30, n_estimators=150, verbose=0)
model.fit(X, y)
joblib.dump(model, '../output/lgbm_model.pkl')

y_pred_train = model.predict(X)
y_pred_test = model.predict(X_test)
y_pred_val = model.predict(X_val)
print(f"Training Mean Squared Error: {round(mean_squared_error(y, y_pred_train),4)}, R-Squared = {round(r2_score(y, y_pred_train),4)}")
print(f"Validation Mean Squared Error: {round(mean_squared_error(y_val, y_pred_val),4)}, R-Squared = {round(r2_score(y_val, y_pred_val),4)}")
print(f"Test Mean Squared Error: {round(mean_squared_error(y_test, y_pred_test),4)}, R-Squared = {round(r2_score(y_test, y_pred_test),4)}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Training Mean Squared Error: 0.0515, R-Squared = 0.9485
Validation Mean Squared Error: 0.0894, R-Squared = 0.9104
Test Mean Squared Error: 0.1233, R-Squared = 0.8624
